In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- Run this at the beginning of EACH Colab session ---
# 2. Define the checkpoint directory (the same one you used for saving)
checkpoint_dir = '/content/drive/MyDrive/your_model_checkpoints'  # Replace with your actual path

# 3. Load Libraries (the libraries from the notebook)
import numpy as np
import pandas as pd
import scipy
import pickle
import random
import os

# loading in and transforming data
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import PIL
from PIL import Image
import imageio
from torchvision.datasets import ImageFolder as IF
from torchsummary import summary

# visualizing data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 4. Transformations, constants, etc
path_monet = '/content/drive/MyDrive/Dataset/trainA'
path_photo = '/content/drive/MyDrive/Dataset/trainB'
img_size = 256
transforms.Compose([
 transforms.Resize((128, 128)), # instead of (256, 256) or larger
 transforms.ToTensor(),
 transforms.Normalize((0.5,), (0.5,)) # Normalize if needed
])

# 5. Set seed
def set_seed(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_seed(42)

# 6. Define your models (VERY IMPORTANT:  Define the ARCHITECTURE before loading!)
# Replace with the actual definitions of G_XtoY, G_YtoX, D_X, D_Y

import torch.nn as nn
import torch.nn.functional as F
class ResidualBlock(nn.Module):

    def __init__(self, conv_dim):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1, instance_norm=True)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1, instance_norm=True)

    def forward(self, x):
        out_1 = F.relu(self.conv1(x))
        out_2 = x + self.conv2(out_1)
        return out_2
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=False, instance_norm=False):

    #Add layers
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    layers.append(conv_layer)

    #Batch normalization
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))

    #Instance normalization
    if instance_norm:
        layers.append(nn.InstanceNorm2d(out_channels))
    return nn.Sequential(*layers)

def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=False, instance_norm=False, dropout=False, dropout_ratio=0.5):
    layers = []
    layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))

    if instance_norm:
        layers.append(nn.InstanceNorm2d(out_channels))

    if dropout:
        layers.append(nn.Dropout2d(dropout_ratio))

    return nn.Sequential(*layers)



class CycleGenerator(nn.Module):

    def __init__(self, conv_dim=64, n_res_blocks=6):
        super(CycleGenerator, self).__init__()

        #Encoder layers
        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=4)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=4, instance_norm=True)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=4, instance_norm=True)

        # Residual Layer
        res_layers = []
        for layer in range(n_res_blocks):
            res_layers.append(ResidualBlock(conv_dim*4))
        self.res_blocks = nn.Sequential(*res_layers)

        # Decoder Layer
        self.deconv4 = deconv(in_channels=conv_dim*4, out_channels=conv_dim*2, kernel_size=4, instance_norm=True)
        self.deconv5 = deconv(in_channels=conv_dim*2, out_channels=conv_dim, kernel_size=4, instance_norm=True)
        self.deconv6 = deconv(in_channels=conv_dim, out_channels=3, kernel_size=4, instance_norm=True)

    def forward(self, x):


        out = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        out = F.leaky_relu(self.conv2(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv3(out), negative_slope=0.2)


        out = self.res_blocks(out)


        out = F.leaky_relu(self.deconv4(out), negative_slope=0.2)
        out = F.leaky_relu(self.deconv5(out), negative_slope=0.2)
        out = torch.tanh(self.deconv6(out))

        return out
class Discriminator(nn.Module):

    def __init__(self, conv_dim=64):
        super(Discriminator, self).__init__()

        #convolutional layers, increasing in depth
        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=4)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=4, instance_norm=True)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=4, instance_norm=True)
        self.conv4 = conv(in_channels=conv_dim*4, out_channels=conv_dim*8, kernel_size=4, instance_norm=True)
        self.conv5 = conv(in_channels=conv_dim*8, out_channels=conv_dim*8, kernel_size=4, batch_norm=True)

        #final classification layer
        self.conv6 = conv(conv_dim*8, out_channels=1, kernel_size=4, stride=1)

    def forward(self, x):


        out = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        out = F.leaky_relu(self.conv2(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv3(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv4(out), negative_slope=0.2)
        out = self.conv6(out)
        return out

G_XtoY = CycleGenerator()
G_YtoX = CycleGenerator()
D_X = Discriminator()
D_Y = Discriminator()

# 7. Load the saved weights
def load_checkpoint(checkpoint_path, map_location=None):
    """
    Load checkpoint
    """
    #model.load_state_dict(torch.load(checkpoint_path))
    checkpoint = torch.load(checkpoint_path, map_location=map_location)
    print(' [*] Loading checkpoint from %s succeed!' % checkpoint_path)
    return checkpoint

# Modify the save_checkpoint function to save to Google Drive by default
def save_checkpoint(iteration, G_XtoY, G_YtoX, D_X, D_Y, checkpoint_dir=checkpoint_dir):
    """
    Saves the parameters of both generators and discriminators.
    """
    #Path
    G_XtoY_path = os.path.join(checkpoint_dir, 'G_XtoY.pkl')
    G_YtoX_path = os.path.join(checkpoint_dir, 'G_YtoX.pkl')
    D_X_path = os.path.join(checkpoint_dir, 'D_X.pkl')
    D_Y_path = os.path.join(checkpoint_dir, 'D_Y.pkl')
    #Saving
    torch.save(G_XtoY.state_dict(), G_XtoY_path)
    torch.save(G_YtoX.state_dict(), G_YtoX_path)
    torch.save(D_X.state_dict(), D_X_path)
    torch.save(D_Y.state_dict(), D_Y_path)

# Load the saved weights from Google Drive
G_XtoY.load_state_dict(load_checkpoint(os.path.join(checkpoint_dir, 'G_XtoY.pkl'),map_location=torch.device('cpu')))
G_YtoX.load_state_dict(load_checkpoint(os.path.join(checkpoint_dir, 'G_YtoX.pkl'),map_location=torch.device('cpu')))
D_X.load_state_dict(load_checkpoint(os.path.join(checkpoint_dir, 'D_X.pkl'),map_location=torch.device('cpu')))
D_Y.load_state_dict(load_checkpoint(os.path.join(checkpoint_dir, 'D_Y.pkl'),map_location=torch.device('cpu')))

print("All models loaded from checkpoints.")

# --- Now you can continue with your training loop or inference ---


 [*] Loading checkpoint from /content/drive/MyDrive/your_model_checkpoints/G_XtoY.pkl succeed!
 [*] Loading checkpoint from /content/drive/MyDrive/your_model_checkpoints/G_YtoX.pkl succeed!
 [*] Loading checkpoint from /content/drive/MyDrive/your_model_checkpoints/D_X.pkl succeed!
 [*] Loading checkpoint from /content/drive/MyDrive/your_model_checkpoints/D_Y.pkl succeed!
All models loaded from checkpoints.


In [ ]:
def reverse_normalize(image, mean_=0.5, std_=0.5):
    if torch.is_tensor(image):
        image = image.detach().numpy()
    un_normalized_img = image * std_ + mean_
    un_normalized_img = un_normalized_img * 255
    return np.uint8(un_normalized_img)


In [ ]:
import os
from PIL import Image

def get_user_image_path():
    image_path = input("Enter the path to your image: ")
    if not os.path.isfile(image_path):
        raise FileNotFoundError(f"File {image_path} not found")
    return image_path

# 2. Image Preprocessing (from search result [3] with torchvision fix)
import torchvision.transforms as transforms

def preprocess_user_image(image_path, image_size=256):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transform(Image.open(image_path).convert('RGB')).unsqueeze(0)

# 3. Model Inference (adapted from search result [4])
def run_inference(model, input_tensor):
    model.eval()
    with torch.no_grad():
        return model(input_tensor)

# 4. Output Handling (from search result [7] with torchvision fix)
def display_or_save_output(output_tensor, save_path=None):
    reverse_transform = transforms.Compose([
        transforms.Normalize(mean=[0., 0., 0.], std=[2, 2, 2]),
        transforms.Normalize(mean=[-0.5, -0.5, -0.5], std=[1, 1, 1]),
        transforms.ToPILImage()
    ])
    result = reverse_transform(output_tensor.squeeze(0).cpu())

    if save_path:
        result.save(save_path)
        print(f"Saved to {save_path}")
    else:
        plt.imshow(result)
        plt.axis('off')
        plt.show()


In [ ]:
# Load pretrained generators (ensure you have these checkpoints)
G_photo_to_monet = load_checkpoint('/content/drive/MyDrive/your_model_checkpoints/G_XtoY.pkl',map_location=torch.device('cpu'))  # From original code

# Interactive workflow
try:
    # Get user input
    img_path = get_user_image_path()

    # Process image
    input_tensor = preprocess_user_image(img_path).to(device)

    # Generate output
    output_tensor = run_inference(G_photo_to_monet, input_tensor)

    # Show/save result
    display_or_save_output(output_tensor)

except Exception as e:
    print(f"Error: {str(e)}")


 [*] Loading checkpoint from /content/drive/MyDrive/your_model_checkpoints/G_XtoY.pkl succeed!
Enter the path to your image: /content/drive/MyDrive/IST3.jpg
Error: name 'device' is not defined


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import torch
from PIL import Image
import torchvision.transforms as transforms

# Assuming CycleGenerator, reverse_normalize, and preprocess_user_image are defined in your previous cells.
# If not, please provide their definitions here.

def convert_to_monet(input_path, output_path):
    # 1. Create an instance of the generator
    generator = CycleGenerator()

    # 2. Load the state dictionary into the generator instance
    generator.load_state_dict(torch.load('/content/drive/MyDrive/your_model_checkpoints/G_XtoY.pkl', map_location=torch.device('cpu')))

    # 3. Preprocess the input image
    input_tensor = preprocess_user_image(input_path).to(torch.device('cpu'))  # Assuming preprocess_user_image is defined

    # 4. Generate the Monet-style image
    generator.eval()
    with torch.no_grad():
        output_tensor = generator(input_tensor)

    # 5. Post-process the output
    output_image = reverse_normalize(output_tensor.cpu().numpy()[0]) # Assuming reverse_normalize is defined
    output_image = np.transpose(output_image, (1, 2, 0))
    output_image = np.clip(output_image, 0, 255).astype(np.uint8)

    # 6. Save the output image
    Image.fromarray(output_image).save(output_path)
    print(f"Image saved to: {output_path}")

# Example usage
convert_to_monet('/content/drive/MyDrive/IST4.jpg', 'monet_output.jpg')

Image saved to: monet_output.jpg
